# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
os.getcwd()

'/Users/christellejoseph/code/Cyber_Tools/IDS'

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
import seaborn as sns

In [6]:
csv_path = '/Users/christellejoseph/code/Cyber_Tools/IDS/data'
file_dir = os.listdir(csv_path)

In [7]:
csv_files = []
for file in file_dir:
    modified_file = file.replace('-WorkingHours', '').replace('-workingHours', '').replace(
                                        '-Afternoon', '').replace('-Morning', '').replace('.pcap_ISCX.csv', '')
    csv_files.append(modified_file)

# Merging: Dictionary Structure

In [10]:
data = {}

for name, filename in sorted(zip(csv_files, file_dir)):
    data[name] = pd.read_csv(os.path.join(csv_path, filename))

# Merging: Single DataFrame

In [ ]:
all_data = pd.concat([pd.read_csv(os.path.join(csv_path, file)) for file in file_dir], ignore_index=True)